## Instituto Tecnológico de Costa Rica
## Escuela de Ingeniería Electrónica
## Maestría en Electrónica
## MP-6122 Reconocimiento de Patrones
## Investigación Corta 2 - Aprendizaje Supervisado
## Douglas Mora Ceciliano
#
### Escogiendo un set de datos:
Para esta investigacion se eligio el set de datos openpowerlifting.csv
Dado su gran tamaño se procede a limpiarlo un poco en Excel previamente se procede a eliminar las columnas que no siguen las reglas estandar: Deadlift4Kg, Bench4Kg, Squat4Kg

Fuente: https://www.kaggle.com/open-powerlifting/powerlifting-database/downloads/powerlifting-database.zip/2#openpowerlifting.csv

El archivo resultante se encuentra en este Github, el mismo se carga a continuacion:

In [615]:
import numpy as np
import pandas as pd
Frame=pd.read_csv('mushroom.csv')

FileNotFoundError: [Errno 2] File b'mushroom.csv' does not exist: b'mushroom.csv'

### Etapa de preprocesado

Con el fin de obtener mejores resultados se procede a preprocesar los datos

#### Determinando tamaño del set de datos
A continuacion se utiliza la funcion shape() para determinar el tamaño de nuestro set de datos.

In [553]:
print (' Filas Columnas')
print (Frame.shape)

 Filas Columnas
(509376, 32)


### Seleccionando los Features

Utilizaremos el codigo head para determinar el nombre de las columnas en nuestro set.

In [554]:
Frame.head()

,Name,Sex,Event,Equipment,Age,AgeClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,...,Place,Wilks,McCulloch,Glossbrenner,IPFPoints,Tested,Federation,Date,MeetCountry,MeetName
0,Daniel McGauley,M,SBD,Raw,NaN,NaN,MR-O,56.1,59,100.0,...,1,286.26,286.26,279.89,383.49,Yes,EPA,4/20/2019,UK,South Midlands Spring Open
1,Rod Ramsey,M,SBD,Raw,NaN,NaN,MR-O,65.7,66,152.5,...,1,396.05,396.05,384.50,579.47,Yes,EPA,4/20/2019,UK,South Midlands Spring Open
2,Terry Martin,M,SBD,Raw,NaN,NaN,MR-O,73.6,74,215.0,...,1,435.06,435.06,421.02,652.83,Yes,EPA,4/20/2019,UK,South Midlands Spring Open
3,Geoff Erzan-Essien,M,SBD,Raw,NaN,NaN,MR-O,76.0,83,135.0,...,1,285.96,285.96,276.54,400.88,Yes,EPA,4/20/2019,UK,South Midlands Spring Open
4,Jason Coultman,M,SBD,Raw,28.5,24-34,MR-O,96.3,105,217.5,...,1,415.79,415.79,400.03,615.97,Yes,EPA,4/20/2019,UK,South Midlands Spring Open


Como se puede observar contamos con un ser de varios datos, pero cual seria una salida interesante de averiguar?
Para este caso se buscara saber si en base a una serie de variables logramos predecir el exito o fracaso para levantar el siguiente peso. Para ello solo utilizaremos el primer ejercicio (Squat).

Para ello seleccionaremos columnas especificas para utilizar como se muestra a continuacion

In [555]:
cols2use=['Sex',#'Equipment',#'Age',
          'BodyweightKg','Squat1Kg','Squat2Kg','Squat3Kg']

In [556]:
Frame=Frame[cols2use]
Frame.head()

,Sex,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg
0,M,56.1,100.0,110.0,120.0
1,M,65.7,152.5,162.5,170.0
2,M,73.6,215.0,-225.0,-225.0
3,M,76.0,135.0,140.0,150.0
4,M,96.3,217.5,225.0,232.5


Estos son entonces los datos de interes, posteriormente se agregaran mas en base a estos mismos. Pero primero se trabajaran un poco mas.

#### Limpiando Nulls
Utilizando el siguiente codigo buscaremos identificar si hay algún valor nulo en el set.

In [557]:
print('Identificacion de cantidad de valores nulos')
nNULL=Frame.isnull().sum()
print(nNULL)

Identificacion de cantidad de valores nulos
Sex               4214
BodyweightKg      4663
Squat1Kg        335150
Squat2Kg        337012
Squat3Kg        341101
dtype: int64


Una vez identificados se procede a eliminar dichos valores nulos

In [558]:
Frame=Frame.dropna()
nNULL=Frame.isnull().sum()
print(nNULL)

Sex             0
BodyweightKg    0
Squat1Kg        0
Squat2Kg        0
Squat3Kg        0
dtype: int64


Este procedimiento nos deja con una cantidad bastante menor de datos. Normalmente Eliminar los valores nulos no deberia ser la primera opcion pero tomando en cuenta el gran numero de datos se decido utilizarla.

In [559]:
print (' Filas Columnas')
print (Frame.shape)

 Filas Columnas
(168045, 5)


Nos quedamos entonces con aproximadamente 149 mil lineas. que representan la seccion de la poblacion para la cual conocemos todos los datos de interes.

#### Definiendo Features y Labels

Una parte fundamental del Aprendizaje supervisado es definir los Features de los Labels. Los primeros son tanto las entradas que utilizamos para probar como para testear nuestro sistema. Mientras que los segundos se usan para entrenar y evaluar el sistema.

Como se menciono anteriormente la idea sera determinar si es posible predecir el exito en el levantamiento 3 basados en un grupo de entradas.

Sexo: Es evidente que por un factor biologico los hombres pueden llegar a levantar mas peso que las mujeres suponiendo igualdad de condiciones por lo que este factor sera tomado en cuenta. Se utilizara 1 para Hombre y 0 para Mujer.

In [560]:
Frame['Sex']=Frame['Sex'].replace({'M':1,'F':0})

Equipo: Parte de la hipotesis es que ciertos equipos pueden afectar el desempeño. Se usara Raw=0, Wraps=1, Multi-ply=2, Single-ply=3

In [561]:
#Frame['Equipment']=Frame['Equipment'].replace({'Raw':0, 'Wraps':1, 'Multi-ply':2, 'Single-ply':3})

Edad: La edad es un factor que tambien sera tomado en cuenta. Se supone que un niño o adulto mayor no contaran con la capacidad fisica que un adulto en sus 20-30 años. Denuevo suponiendo igualdad de condiciones.

Peso: Se estima que el peso corporal debe estar relacionado con la cantidad de peso que se puede levantar.

Exito en Squat 1: Se tuvo exito en el levantamiento 1. 1 es un Exito y 0 es un Fallo.

In [562]:
Frame['Success1']=Frame['Squat1Kg'].apply(lambda x: 1 if x>=0 else 0)

Squat1: Cantidad de peso levantado en el primer squat (Se aplica valor absoluto para separar el valor levantado del signo que era la representacion del exito o el fracaso)

In [563]:
Frame['Squat1Kg']=Frame['Squat1Kg'].abs()

Se hace lo mismo con el Squat2 y Squat3

In [564]:
Frame['Success2']=Frame['Squat2Kg'].apply(lambda x: 1 if x>=0 else 0)
Frame['Squat2Kg']=Frame['Squat2Kg'].abs()
Frame['Success3']=Frame['Squat3Kg'].apply(lambda x: 1 if x>=0 else 0)
Frame['Squat3Kg']=Frame['Squat3Kg'].abs()

El resultado se observa a continuacion:

In [565]:
Frame.head()

,Sex,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Success1,Success2,Success3
0,1,56.1,100.0,110.0,120.0,1,1,1
1,1,65.7,152.5,162.5,170.0,1,1,1
2,1,73.6,215.0,225.0,225.0,1,0,0
3,1,76.0,135.0,140.0,150.0,1,1,1
4,1,96.3,217.5,225.0,232.5,1,1,1


Se selecciona entonces los Features de interes.

In [566]:
Features = Frame[['Sex'#,'Equipment','Age'
                  ,'BodyweightKg','Squat1Kg','Success1','Squat2Kg','Success2','Squat3Kg']]

El Label sera la caracteristica a predecir.

In [567]:
Label = Frame['Success3']

### Analisis a partir de multiples metodos supervisados:

Se escogieron 6 metodos de aprendizaje supervisado. Para comparar sus propiedades a continuacion se muestran los resultados obtenidos para cada uno de ellos.

Antes de entrar en materia es importante comentar una caracteristica particular de este tipo de metodos es que se requiere un grupo de datos para entrenar y un grupo de datos para Testear que el entrenamiento se ha llevado a cabo con exito. Para ello utilizaremos la siguiente instruccion:

In [568]:
from sklearn.model_selection import train_test_split

Separaremos entonces nuestros features y labels en dos grupos uno de test y uno de train

In [612]:
X_train, X_test, Y_train, Y_test = train_test_split(Features, Label, random_state=13)

In [613]:
from sklearn.ensemble import DecisionTreeClassifier

ImportError: cannot import name 'DecisionTreeClassifier' from 'sklearn.ensemble' (C:\Users\moradoug\AppData\Local\Continuum\miniconda3\envs\Inv_Corta_2.ipynb\lib\site-packages\sklearn\ensemble\__init__.py)

In [614]:
DecisionTreeModel = DecisionTreeClassifier(max_depth=7)

In [608]:
DecisionTreeModel.fit(X_train,Y_train)

C:\Users\moradoug\AppData\Local\Continuum\miniconda3\envs\Inv_Corta_2.ipynb\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=120, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [609]:
from sklearn.metrics import accuracy_score

In [610]:
Y_predict=DecisionTreeModel.predict(X_test)

In [611]:
accuracy_score(Y_test,Y_predict)

0.6032086070646482

In [449]:
from sklearn.tree import export_graphviz
export_graphviz(DecisionTreeModel, out_file='simple_tree.dot', feature_names=Features.columns,impurity=False,class_names=['Not survived','Survived'],filled=True)
from subprocess import call
call(['dot', '-T', 'png', 'simple_tree.dot', '-o', 'simple_tree.png'])

FileNotFoundError: [WinError 2] The system cannot find the file specified